<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/figure_generation/GenFig3_S20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Generates figure 3 and supplementary figure 20**

This notebook generates figures for showing the improvement of BUTTERFLY correction on scRNA-Seq data, by comparing a downsampled and full dataset. Furthermore, we show the effect of "borrowing" CU histogram information from similar datasets, and determine the sampling noise, which sets a theoretical maximum performance for the prediction in downsampling scenarios such as this.

Steps:
1. Download the code and processed data
2. Setup the R environment
3. Generate the figures


**1. Download the code and processed data**

In [1]:
#download the R code
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 944 (delta 35), reused 28 (delta 14), pack-reused 883
Receiving objects: 100% (944/944), 7.37 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (589/589), done.


In [ ]:
#download processed data from Zenodo for all datasets
![ -d "figureData" ] && rm -r figureData
!mkdir figureData
!cd figureData && wget https://zenodo.org/record/3909758/files/FigureData.zip?download=1 && unzip 'FigureData.zip?download=1' && rm 'FigureData.zip?download=1'

!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_3.zip?download=1 && unzip 'PBMC_V3_3.zip?download=1' && rm 'PBMC_V3_3.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3_2.zip?download=1 && unzip 'PBMC_V3_2.zip?download=1' && rm 'PBMC_V3_2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V3.zip?download=1 && unzip 'PBMC_V3.zip?download=1' && rm 'PBMC_V3.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG.zip?download=1 && unzip 'PBMC_NG.zip?download=1' && rm 'PBMC_NG.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/PBMC_V2.zip?download=1 && unzip 'PBMC_V2.zip?download=1' && rm 'PBMC_V2.zip?download=1'
!cd figureData && wget https://zenodo.org/record/3909758/files/EVALPBMC.zip?download=1 && unzip 'EVALPBMC.zip?download=1' && rm 'EVALPBMC.zip?download=1'


--2020-07-02 17:08:04--  https://zenodo.org/record/3909758/files/FigureData.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4901056 (4.7M) [application/octet-stream]
Saving to: ‘FigureData.zip?download=1’

FigureData.zip?down 100%[===================>]   4.67M  5.26MB/s    in 0.9s    

2020-07-02 17:08:06 (5.26 MB/s) - ‘FigureData.zip?download=1’ saved [4901056/4901056]

Archive:  FigureData.zip?download=1
  inflating: extreme_genes.txt       
 extracting: Fig1_h1.RDS             
 extracting: Fig1_h2.RDS             
 extracting: Fig1_r1_III.RDS         
 extracting: Fig1_r2_III.RDS         
  inflating: Fig3_ldata.RDS          
  inflating: Fig3_ldata2.RDS         
  inflating: Fig4_d1.RDS             
  inflating: PBMC_V3_3_ds10_20Times.RData  
--2020-07-02 17:08:07--  https://zenodo.org/record/3909758/files/PBMC_V3_3.zip?download=1
Reso

In [ ]:
#Check that download worked
!cd figureData && ls -l && cd PBMC_V3_3 && ls -l

**2. Prepare the R environment**

In [ ]:
#switch to R mode
%reload_ext rpy2.ipython


In [ ]:
#install the R packages and setup paths
%%R
install.packages("dplyr")
#install.packages("preseqR")
install.packages("ggplot")
install.packages("DescTools")
install.packages("ggpubr")



**3. Define a general function to generate any of the figures S7-S19 save it to disk**

This function creates a prediction evaluation figure for a dataset and saves it to file. We show the following five methods.

1. Preseq DS (Rational functions approximation), trunkating CU histograms at 2
2. Preseq DS, trunkating CU histograms at 20
3. Zero-trunkated negative binomial (ZTNB)
4. "Best practice", which selects Preseq DS (here with histograms trunkated at 2) if the number of copies per molecule CV > 1, otherwise ZTNB.
5. Scaling only - scales all genes equally, i.e. makes no advanced prediction

The plots are based on a loess fit of the prediction errors.

In [ ]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [ ]:
#Import the code for prediction (available in other notebooks)
%%R
source(paste0(sourcePath,"ButterflyHelpers.R"))
#source(paste0(sourcePath,"preseqHelpers.R"))
source(paste0(sourcePath,"CCCHelpers.R"))
source(paste0(sourcePath,"ggplotHelpers.R"))





In [ ]:
#create figure directory
![ -d "figures" ] && rm -r figures
!mkdir figures

In [ ]:
#Create and save the figures
%%R
library(ggplot2)
library(ggpubr)
library(hexbin)
library(dplyr)


ldata = readRDS(paste0(figure_data_path, "Fig3_ldata.RDS"))
ldata2 = readRDS(paste0(figure_data_path, "Fig3_ldata2.RDS"))



#generate plot data
plotdata = tibble(gene=ldata$gene, 
                  x=ldata$x, 
                  nopred=ldata$nopred - ldata$trueval,
                  pred=ldata$pred - ldata$trueval,
                  poolpred=ldata$poolpred - ldata$trueval)

#melt
plotdata.m = reshape2::melt(plotdata, id.vars=c("gene","x"), measure.vars = c("nopred", "pred", "poolpred"))

labl = labeller(variable = 
                      c("nopred" = "No Correction",
                        "pred" = "Correction",
                        "poolpred" = "Correction using Pooling"))

dfline = data.frame(x=c(0,16), y=c(0,0))

dummyData = data.frame(x=c(0,0), y=c(1.1, -1.5)) #used in a trick to set y axis range below

fig3 = ggplot(plotdata.m) +
  stat_binhex(bins=60,na.rm = TRUE, mapping=aes(x = x, y=value, fill = log(..count..))) + # opts(aspect.ratio = 1) +
  facet_wrap(facets = ~  variable, scales = "free_x", labeller = labl, ncol=3) +
  geom_line(data=dfline, mapping = aes(x=x, y=y), color="black", size=1) + 
  geom_blank(data = dummyData, mapping = aes(x=x, y=y)) + #trick to set y axis range
  labs(y=expression(Log[2]*" fold change (CPM)"), x=expression(Log[2]*"(CPM + 1)")) +
  theme(panel.background = element_rect("white", "white", 0, 0, "white"),
        legend.position= "bottom", legend.direction = "horizontal",#, legend.title = element_blank())
        strip.text.x = element_text(size = 12, face = "bold"),
        #legend.position= "none",
        strip.background = element_blank())

fig3 # for some reason this plot sometimes fail and show an error ("hbin" ...) - Restart R and try again in that case


ggsave(
  paste0(figure_path, "Fig3.png"),
  plot = fig3, device = "png",
  width = 7, height = 4, dpi = 300)


#########################
# Fig S20 (Sampling noise)
#########################
#cpm and log transform

plotdata2 = tibble(gene=ldata2$gene, 
                  x=ldata2$x, 
                  y=ldata2$sampling - ldata$nopred)


dfline = data.frame(x=c(0,16), y=c(0,0))
dummyData = data.frame(x=c(0,0), y=c(1.1, -1.5))

figS20 = ggplot(plotdata2) +
  stat_binhex(bins=60,na.rm = TRUE, mapping=aes(x = x, y=y, fill = log(..count..))) + # opts(aspect.ratio = 1) +
  #facet_wrap(facets = ~  variable, scales = "free_x", labeller = labl, ncol=3) +
  geom_line(data=dfline, mapping = aes(x=x, y=y), color="black", size=1) + 
  geom_blank(data = dummyData, mapping = aes(x=x, y=y)) + #trick to set y axis range
  labs(y=expression(Log[2]*" fold change (CPM)"), x=expression(Log[2]*"(CPM + 1)")) +
  theme(panel.background = element_rect("white", "white", 0, 0, "white"),
        legend.position= "bottom", legend.direction = "horizontal",#, legend.title = element_blank())
        strip.text.x = element_text(size = 12, face = "bold"),
        #legend.position= "none",
        strip.background = element_blank())

figS20 # for some reason this plot sometimes fail and show an error ("hbin" ...) - Restart R and try again in that case


ggsave(
  paste0(figure_path, "FigS20.png"),
  plot = figS20, device = "png",
  width = 3, height = 4, dpi = 300)



#The data to present over the plots


print(paste0("CCC, no pred: ", getCCC(ldata$nopred, ldata$trueval))) #0.981275291888894
print(paste0("CCC, pred no pooling: ", getCCC(ldata$pred, ldata$trueval))) #0.993829998877551
print(paste0("CCC, pred with pooling: ", getCCC(ldata$poolpred, ldata$trueval))) #0.997028015743896
print(paste0("CCC, no pred, ds 100 times vs ds: ", getCCC(ldata2$nopred, ldata2$sampling))) #0.99890437562339





